## ENERGIA HIDRAULICA. CASO PRACTICO 1

Se propone el ejemplo de una central hidroeléctrica fluyente de una potencia instalada de 50 MW, con una turbina que presenta un rendimiento del 92 % y un generador con un rendimiento del 97 %. El río que abastece dicha central tiene un Qmedio= 60 m3/s, debido a una precipitación media en la región de 600 mm y una escorrentía del 0,4.

La captura de agua se realiza con un pequeño azud situado a 200 m sobre el nivel del mar (msnm), y después se hace conducir por un canal rectangular de hormigón de 5 m de alto por 12 m de ancho, dejando una altura libre de agua de 0,5 m.

La longitud del canal es de 2,5 km con una pendiente del 0,01 %. Las pérdidas localizadas por la toma y las rejas son de 1 m. A continuación, el agua se conduce por dos tuberías forzadas de acero durante 500 m, en donde el agua circula a una velocidad de 5 m/s. El cambio de sección del canal a la tubería genera unas pérdidas localizadas de 0,5 m.

(Datos adicionales: coeficiente de hormigón = 0,014; coeficiente acero 0,012).

Se debe calcular:

1. El área de aportación hídrica de la central.
2. La velocidad del agua, las pérdidas totales y el caudal en el canal.
3. Las pérdidas totales en la tubería.
4. Las pérdidas totales de la instalación.
5. El salto neto, el salto bruto y el nivel de restitución.
6. La potencia a la salida de la turbina y el tipo de turbina más adecuada.

El campo de uso de las turbinas hidráulicas según altura, caudal y potencia se muestra en la figuras siguientes:

![image info](https://upload.wikimedia.org/wikipedia/commons/thumb/e/ea/Turbinas.svg/1280px-Turbinas.svg.png)

Fuente: [Wikipedia, turbinas hidráulicas](https://es.wikipedia.org/wiki/Turbina_hidr%C3%A1ulica#/media/Archivo:Turbinas.svg)

![Caudal_vs_salto](Caudal_vs_salto.png)
*Fuente: Escher Wiss*

La resolución de este caso la haremos empleando el lenguaje Pyhton. Para la realización de algunas operaciones matemáticas vamos a apoyarnos en la herramienta Numpy.

In [7]:
import numpy as np

## 1. Area de aportación hídrica

El cálculo del aporte de la central hidroeléctrica viene dado por la siguiente ecuación:

$ AportacionCentral = PrecipitacionMedia * Escorrentia * Superfice $

El área o superficie de aportación se obtiene despejando la ecuación anterior:

$Superfice = {AportacionCentral \over {PrecipitacionMedia * Escorrentia}}$

Contamos con los siguientes datos de precipitación media en mm o l/m2, de escorrentía y de caudal medio en m3/s:

In [1]:
Prec_med = 600 # mm o l/m2
Escorr = 0.4
Qmed = 60 #m3/s

Vamos a convertir la precipitación media a Hm3/Km2. El aporte a la central será la precipitación media expresada en la cantidad de Hm3 caídos en un año $AporteCentral = Qmedio . 3600 \frac{seg} {hora} . 24 \frac{horas} {día}. 365 \frac{días} {año}$:

In [2]:
Prec_med = Prec_med / 1000000000 # Hm3/m2
Prec_med = Prec_med * 1000000 # Hm3/Km2
print("Prec_med: ",Prec_med," Hm3/Km2")

Aporte = Qmed * 3600 * 24 * 365 # En un año. m3/año
Aporte = Aporte / 1000000 # Conversión a Hm3
print("Aportación de central: ",Aporte," Hm3/año")

Prec_med:  0.6  Hm3/Km2
Aportación de central:  1892.16  Hm3/año


In [3]:
Area = Aporte / (Prec_med * Escorr)
print("Area de aportación hídrica:",Area,"Km2")

Area de aportación hídrica: 7884.000000000001 Km2


Nuestro área de aportación a la central hidroeléctrica en este caso es de **7.884 Km2**

## 2. Velocidad del agua, pérdidas totales y caudal en el canal

La ecuación de Manning describe la pérdida que se producen en los canales o tuberías por donde circula el agua y se expresa en unidad de longitud, metros en este caso:

$$ i = \frac{n^{2} * v^{2}} {Rh^{\frac{4} {3}}}$$

Donde:

i: Pérdida de carga, en m, o pendiente del canal en m/m  
n: Coeficiente de material. 0,012 para el acero y 0,014 para el hormigón  
v: Velocidad de flujo, en m/s.  
Rh: Radio hidráulico, en m.

De entrada tenemos el dato de pendiente del (i), que está representado en porcentaje, el cual se transforma en m/m diviviendo entre 100. Se nos indica que el canal por donde circula el agua es de hormigón, del cual conocemos su coeficiente (n). 

In [4]:
ic = 0.01/100 # m
nc = 0.014 # hormigón

En cuanto al radio hidráulico, se nos dice que el agua circula por un canal rectangular de hormigón. El radio hidráulico en este caso de obtiene del cociente entre el área de sección transversal y su perímetro mojado.

$$R_hc = \frac{Area_{sección}} {Perímetro_{mojado}}$$

Como la sección es de forma rectangular, la anterior relación se expresa de la siguiente forma:

$$R_hc = \frac{b . h_{agua}} {b + 2 . h_{agua}}$$

Donde:  
b: Ancho de base, en m.  
h_agua: Altura de la columna de agua, en m. Equivalente a la diferencia entre la altura del canal y la libre de agua

Estos datos los conocemos, por lo que podemos calcular el radio de flujo

In [5]:
Altura = 5 # Altura total del canal, en m
Ancho = 12 # Ancho de base del canal, en m
Alt_libre = 0.5 # Altura libre de agua, en m
h = Altura - Alt_libre # Altura mojada por el agua, en m

Rhc = (Ancho * h) / (Ancho + 2*h) #Radio hidráulico de canal, en m
print("Radio hidráulico de canal:",Rhc,"m")

Radio hidráulico de canal: 2.5714285714285716 m


Nos queda determinar la velocidad del agua. Despejando la ecuación de Manning tenemos que:

$$ v = \sqrt{\frac{i * Rh^{\frac{4} {3}} } {n^{2}}}$$

In [9]:
vc = np.power((ic * np.power(Rhc,(4/3))/ np.power(nc,2)),(1/2))
print("Velocidad del agua en el canal:",vc,"m/s")

Velocidad del agua en el canal: 1.3406691244554763 m/s


Con los datos del canal y su pendiente pendiente tenemos que la velocidad con la que circula el agua es de **1,34 m/s**

Las pérdidas totales vienen dadas por la suma de las pérdidas primarias y secundarias

$$ PérdidasTotales = PérdidasPrimarias + PérdidasSecundarias$$

Las pérdidas primarias son debidas a la fricción del agua sobre la superficie del canal a lo largo de éste. Se traduce en:

$$PérdidasPrimarias = Longitud * i$$

Las pérdidas secundarias en este caso se producen al pasar por las tomas y las rejillas. Ese dato lo tenemos

In [11]:
Lc = 2500 # Longitud del canal, en m
PerdSecc = 1 # m
Perd_can = Lc * ic + PerdSecc
print("Pérdidas totales en canal:",Perd_can,"m")

Pérdidas totales en canal: 1.25 m


Tenemos unas pérdidas totales de **1,25 m**

Finalmente procedemos a calcular el caudal del canal (Q), que resulta del producto del área de la sección transversal (S) del canal por la velocidad de flujo:

$$Q = S * v $$
Si desglosamos la fórmula del área, en función de la forma rectangular de la sección mojada tenemos que:
$$Q = b * h_{agua} * v $$

Donde b es el ancho de la base del canal y $h_{agua}$ es la altura de la columna de agua

In [12]:
Q = Ancho * h * vc
print("Caudal del canal:",Q,"m3/s")

Caudal del canal: 72.39613272059572 m3/s


Dadas las características de este canal, su caudal es de **72,396 m3/s**

## 3. Pérdidas totales en la tubería

Nuevamente recurrimos a la fórmula de pérdidas totales que resulta de la suma de las pérdidas primarias y secundarias. Como dato se nos suministra el de las pérdidas secundarias al cambio de régimen de canal a tuberías, equivalente a 0,5m. 

Falta estimar las pérdidas primarias. Como dato tenemos que se trata de dos tuberías forzadas por las cuales el agua recorre 500m. Se infiere que son de secciones circulares y que están hechos de acero. Es decir, sus coeficientes de material son 0,012. Para calcular la pérdida de carga (i) de este caso recurrimos nuevamente a la ecuación de Manning, donde conocemos el coeficiente de material de las tuberías, la velocidad se nos suministra como dato y nos falta conocer el radio hidráulico de las tuberías. 

Hemos dicho que el radio hidráulico resulta del cociente entre la sección mojada y su perímetro. La sección mojada de las tuberías la podemos inferir porque estimamos previamente el caudal y conocemos la nueva velocidad de flujo:

$$S_t = {Q \over v_t}$$

Donde S_t es el área total de las seciones de tubería, Q es el caudal de agua que pasa tanto por el canal como por las tuberías y v_t es la velocidad de flujo por las tuberías, dato suministrado en el enunciado.

In [13]:
vt = 5 # velocidad de flujo en tubería, en m/s
St = Q / vt # Area total de secciones de tuberías, en m2

Al tratarse de dos tuberías circulares, podemos obtener el área unitaria de cada una dividiendo el área total entre 2, $A_t = {S_t \over 2}$, y posteriormente deducir el radio, $r = \sqrt{A_t \over \pi}$, que nos permitirá sucesivamente estimar el perímetro mojado de cada tubería, $Pt = 2.\pi.r$

In [14]:
At = St/2 # Area uniaria de cada tuberia, en m2
r = np.power((At/np.pi),0.5) #radio de cada tubería, en m
Pt = 2*np.pi*r # Perímetro mojado de cada tubería, en m
print("Area unitaria:",At,"m2. ","Radio de tubería:",r,"m. ","Perímetro mojado de tubería:",Pt,"m.")

Area unitaria: 7.239613272059572 m2.  Radio de tubería: 1.5180383646811904 m.  Perímetro mojado de tubería: 9.538116348699782 m.


El radio hidráulico de cada tubería, como recordamos viene dado de la fórmula $R_{ht} = {A_t \over P_t}$, el cual podemos estimar porque ya conocemos las variables de área y perímetro

In [15]:
Rht = At / Pt # Radio hidráulico de tubería
print("Radio hidráulico de tubería:",Rht,"m")

Radio hidráulico de tubería: 0.7590191823405952 m


Conociendo el radio hidráulico de cada tubería forzada, podemos calcular la pérdida de carga asociada (it) con la ecuación de Manning:
$$i_t = {n_t^{2} * v_t^{2} \over {R_{ht}^{\frac{4} {3}}}}$$ 

In [16]:
nt = 0.012 # Coeficiente de acero
it = np.power(nt*vt,2)/np.power(Rht,4/3)
print("Pérdida de carga de tubería",it,"m")

Pérdida de carga de tubería 0.0051995468822850395 m


Una vez conocida esta pérdida de carga y conociendo como dato de entrada la distancia (Lt) recorrida por estas tuberías procedemos a estimar las pérdidas primarias de las tuberías $PPrim_t = i_t*L_t$. Seguidamente sumamos éstas con las secundarias para obtener las pérdidas totales de las tuberías $Perd_t = PPrim_t + PSec_t$

In [17]:
Lt = 500 # Longitud de tuberías, en m
PPrimt = it*Lt # Pérdidas primarias en tuberías, en m
PSect = 0.5 # Pérdidas secundarias en tuberías, en m
Perd_tub = PPrimt + PSect
print("Pérdidas totales en tuberías:",Perd_tub,"m")

Pérdidas totales en tuberías: 3.0997734411425197 m


Las pérdidas totales en tubería están en el orden de **3,1m**

## 4. Pérdidas totales de la instalación

Para conococer las pérdidas totales en la instalación sumamos las pérdidas totales del canal más las de las tuberías
$$PérdidasTotales_{instalacion} = PérdidasTotales_{canal} + PérdidasTotales_{tuberías}$$

In [34]:
PerdTotales = Perd_can + Perd_tub
print("Pérdidas totales en instalación",PerdTotales,"m")

Pérdidas totales en instalación 4.34977344114252 m


Las pérdidas totales en esta instalación son del orden de **4,35m**

## 5. Salto neto, salto bruto y nivel de restitución

El salto neto está directamente relacionado con las variables con el caudal Q, la gravedad terrestre, la densidad de fluido $\rho$ y el rendimiento global de los equipos para determinar la potencia disponible de la central:

$$P = Q * g * \rho * H_n * e$$

Donde:  
$P$: Potencia disponible, en vatios (W). Dato que tenemos de entrada, que como está en MW, hay que multiplicar por un millón para convertir a W.  
$Q$: Caudal en el sistema. Este dato lo estimamos en el segundo enunciado.  
$g$: Gravedad terrestre, asumimos el valor 9,81 $m/s^2$.  
$\rho$: Densidad de fluido, en $kg/m^3$. Asumimos para el agua 1000 $kg/m^3$.   
$e$: Rendimiento global, resultado del producto del rendimiento de cada equipo $\eta$  
$H_n$: Salto neto, variable a despejar y calcular

$$H_n = {P \over Q * g * \rho *  e}$$

In [31]:
P = 50 * 1000000 # Potencia disponible, en MW
g = 9.81 # Gravedad terrestre, en m/s2
den = 1000 # Densidad del agua, en kg/m3

Para obtener e rendimiento global multiplicamos los rendimientos de la turbina y del generador, 92% y 97%, respectivamente: $e = \eta_{turb} . \eta_{gen}$. Si tuviéramos datos del rendimiento del transformador, también lo incluiríamos en esta operación de productos.

In [18]:
e_turb = 0.92
e_gen = 0.97
E = e_turb * e_gen
print("Rendimiento global:",E*100,"%")

Rendimiento global: 89.24 %


Una vez contamos con el rendimiento global del sistema, de 89,24% en este caso, contamos con todas las variables necesarias para estimar el salto neto $H_n$:

In [32]:
Hn = P / (Q * g * den * E)
print("Salto neto:",Hn,"m")

Salto neto: 78.89074690614001 m


El salto neto resultante es de **78,89 m**

Normalmente para estimar el salto neto $H_n$ partimos del salto bruto, $H_b$, que suele ser un valor inicial conocido, al cual se le restan las pérdidas totales en el sistema, equivalentes a metros desaporvechados $H_n = H_b - Perd_t$. Pero dado que hicimos otra aporximación al problema estimando las pérdidas y el salto neto, podemos estimar el salto bruto despejando la anterior ecuación.
$$H_b = H_n + Perd_t$$

In [35]:
Hb = Hn + PerdTotales
print("Salto bruto:",Hb,"m")

Salto bruto: 83.24052034728253 m


Tenemos entonces que el salto bruto es **83,24 m**, pero debido a las pérdidas, tendremos un salto neto aprovechable de 78,89m.

El nivel de descarga corresponde a la altura a la cual descargamos el caudal de agua de vuelta al río luego de ser turbinada. Está relacionado con la cota del azud o de la toma de agua y el salto bruto $Hb$: $NivelDescarga = CotaToma - H_b$. Se nos informa que la captación de agua se hace por medio de un azud a 200 m.s.n.m.m. (metros sobre el nivel medio del mar) y el salto bruto lo calculamos previamente.

In [36]:
Cota = 200 # Cota de captación de agua s.n.m., en m.
NivDesc = Cota - Hb
print("Nivel de descarga",NivDesc,"")

Nivel de descarga 116.75947965271747 


Por lo tanto calculamos que el nivel de descarga del agua turbinada se sitúa a **116,76 m.s.n.m** (metros sobre el nivel medio del mar).

## 6. Potencia a la salida de la turbina y tipo de turbina más adecuada

Si recordamos la ecuación de potencia en función del caudal, gravedad, densidad de agua, salto neto y rendimiento(e), tenemos que este último término resulta del producto del rendimiento de cada elemento:
$$P = Q * g * \rho * H_n * \eta_{turb} * \eta_{gen}$$

Para calcular la potencia a la salida de la turbina podemos tomar dos caminos:

1. Descartar el rendimiento del generador y calcular la potencia de la turbina con los demás términos $P_{turb} = Q * g * \rho * H_n * \eta_{turb}$

In [39]:
P_turb_1 = Q * g * den * Hn * e_turb /1000
print("Potencia de turbina",P_turb_1, "KW")

Potencia de turbina 51546.39175257733 KW


2. Despejar la ecuación de la potencia del sistema para dividir dicha potencia que conocemos de antemano entre el rendimiento del generador, con lo cual obtendríamos la potencia de la turbina por separado 
$P = P_{turb} *  \eta_{gen} => P_{turb} = {P \over \eta_{gen}}$ 

In [40]:
P_turb_2 = (P / e_gen) / 1000
print("Potencia de turbina",P_turb_2, "KW")

Potencia de turbina 51546.39175257732 KW


Por cualquiera de los dos caminos tomados, tenemos que la potencia a la salida de la turbina es de **51,55 MW**, que luego al pasar por el generador, la potencia queda en 50 MW.

Nos queda escoger la turbina más adecuada para nuestro sistema. Sabemos que nuestro caudal es de 72,4 m3/s, que el salto neto es de 78,9 m y que la potencia del sistema es de 50MW.

Comparamos las gráficas de aplicación de turbinas, tanto la de Wikipedia como la de Escher Wiss
![Tipo_turbina](3-Caso1-tipos-turbina.jpg)

La gráfica de Wikipedia nos sugiere una tipo Francis y las de Escher Wiss una turbina diagonal. PAra efectos prácticos de este ejercicio conluimos que para nuestro sistema nos conviene una **turbina tipo Francis**.